In [2]:
import pandas as pd
import numpy as np
from espandas import Espandas

In [3]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [1]:
from pd_ela import *

In [2]:
es

<Elasticsearch([{'host': '35.194.152.180'}])>

In [3]:
es = connect_ela(Host='localhost')

In [4]:
es

<Elasticsearch([{'host': 'localhost'}])>

# Api test

In [11]:
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="t-index", doc_type='t', id=1, body=doc)
print(res['result'])

created


In [183]:
empty = {
}
res = es.index(index="t", doc_type='e', body=empty)
print(res['result'])

created


In [184]:
es.delete(index="t",doc_type="e",id=1)

{'_index': 't',
 '_type': 'e',
 '_id': '1',
 '_version': 2,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

# bulk api check

In [14]:
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

In [15]:
df.head()

,A,B,C,D
0,32,63,71,69
1,62,83,78,77
2,15,53,84,86
3,46,39,15,51
4,42,61,44,82


In [42]:
INDEX="dataframe"
TYPE= "record"

In [43]:
def rec_to_actions(df):
    import json
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record, default=int))

In [20]:
tes = rec_to_actions(df.head())
while True:
    try:
        print(next(tes))
    except StopIteration:
        print('sropiteration')
        break

{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 32, "B": 63, "C": 71, "D": 69}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 62, "B": 83, "C": 78, "D": 77}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 15, "B": 53, "C": 84, "D": 86}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 46, "B": 39, "C": 15, "D": 51}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 42, "B": 61, "C": 44, "D": 82}
sropiteration


In [41]:
create("dataframe", "record", Host='35.194.152.180')

created


In [44]:
if not es.indices.exists(INDEX):
    raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%INDEX)

r = es.bulk(rec_to_actions(df)) # return a dict

if r["errors"] == True:
    print('error happens because of {}'.format(r['items'][0]['index']['error']))
else:
    print('succeeds')

succeeds


In [36]:
r['items'][0]['index']['error']

{'type': 'cluster_block_exception',
 'reason': 'blocked by: [FORBIDDEN/12/index read-only / allow delete (api)];'}

In [40]:
r

{'took': 0,
 'errors': True,
 'items': [{'index': {'_index': 'dataframe',
    '_type': 'record',
    '_id': 'sZmsUmYB3n6arYnVEqbq',
    'status': 403,
    'error': {'type': 'cluster_block_exception',
     'reason': 'blocked by: [FORBIDDEN/12/index read-only / allow delete (api)];'}}},
  {'index': {'_index': 'dataframe',
    '_type': 'record',
    '_id': 'spmsUmYB3n6arYnVEqbq',
    'status': 403,
    'error': {'type': 'cluster_block_exception',
     'reason': 'blocked by: [FORBIDDEN/12/index read-only / allow delete (api)];'}}},
  {'index': {'_index': 'dataframe',
    '_type': 'record',
    '_id': 's5msUmYB3n6arYnVEqbq',
    'status': 403,
    'error': {'type': 'cluster_block_exception',
     'reason': 'blocked by: [FORBIDDEN/12/index read-only / allow delete (api)];'}}},
  {'index': {'_index': 'dataframe',
    '_type': 'record',
    '_id': 'tJmsUmYB3n6arYnVEqbq',
    'status': 403,
    'error': {'type': 'cluster_block_exception',
     'reason': 'blocked by: [FORBIDDEN/12/index read-onl

In [43]:
df = (100 * pd.DataFrame(np.round(np.random.rand(100, 5), 2))).astype(int)
df.columns = ['A', 'B', 'C', 'D', 'E']
df['indexId'] = (df.index + 100).astype(str)

In [44]:
INDEX = 'foo_index'
TYPE = 'bar_type'
esp = Espandas()

In [45]:
esp.es_write(df, INDEX, TYPE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/espandas/espandas.py:85: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(sorted(df.columns), axis=1)


In [90]:
def pd_elastic(df):
    from espandas import Espandas
    df['indexId'] = (df.index).astype(str)
    INDEX = input('index name: ')
    TYPE = input('type name: ')
    esp = Espandas()
    esp.es_write(df, INDEX, TYPE)

In [24]:
df = pd.read_csv('f100.csv')
f100_t = df.head(15)
f100_t['indexId'] = (f100_t.index).astype(str)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
INDEX = 'f100_t_index'
TYPE = 'f100_t_type'
esp = Espandas()
esp.es_write(f100_t, INDEX, TYPE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/espandas/espandas.py:85: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(sorted(df.columns), axis=1)


In [23]:
help(esp.es_write)

Help on method es_write in module espandas.espandas:

es_write(df, index, doc_type, index_name='indexId') method of espandas.espandas.Espandas instance
    Insert a Pandas DataFrame into ElasticSearch
    :param df: the DataFrame, must contain the column 'indexId' for a unique identifier
    :param index: the ElasticSearch index
    :param doc_type: the ElasticSearch doc_type



In [29]:
type(f100_t.index)

pandas.core.indexes.range.RangeIndex

In [46]:
df = pd.read_csv('rsi_t.csv')

In [84]:
df.head()

,es_date,raw_date,btc_volume,btc_price,RSI_6,RSI_9,RSI_15,indexId
0,2010-07-24T00:00:00Z,20100724,10400.000000,0.053,NaN,NaN,NaN,0
1,2011-03-01T00:00:00Z,20110301,79956.000000,0.930,NaN,NaN,NaN,1
2,2011-06-04T00:00:00Z,20110604,0.001000,17.830,NaN,NaN,NaN,2
3,2011-08-08T00:00:00Z,20110808,10770.515208,8.890,NaN,NaN,NaN,3
4,2012-06-22T00:00:00Z,20120622,0.031337,6.490,NaN,NaN,NaN,4


In [81]:
df['indexId'] = (df.index).astype(str)
INDEX = 'rsi_index'
TYPE = 'rsi_type'
esp = Espandas()
esp.es_write(df, INDEX, TYPE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/espandas/espandas.py:85: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(sorted(df.columns), axis=1)


In [1]:
def esbulk(Index, Type, df, Host='35.194.152.180'):
    es = Elasticsearch([Host], timeout=30)
    if not isinstance(df, pd.DataFrame):
        raise ValueError('df must be a pandas DataFrame')        

    def rec_to_actions(df):
        for record in df.to_dict(orient="records"):
            yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (Index, Type))
            yield (json.dumps(record, default=int))
    if not es.indices.exists(Index):
        raise RuntimeError('index does not exists')
    
#     r = es.bulk(rec_to_actions(df)) # return a dict
#     write('not r["errors"]: {}'.format(not r["errors"]))
    
    try:
        r = es.bulk(rec_to_actions(df)) # return a dict
        write('not r["errors"]: {}'.format(not r["errors"]))
    except Exception as e:
        write('An unknown error occured connecting to ElasticSearch: %s' % e)

# query

In [112]:
es = Elasticsearch(['35.194.152.180:9200'], timeout=30)

In [121]:
doc = {'size':10000, 'query':{"match_all": {}}}

In [216]:
res1 = es.search(index="btc_top100_index", body=doc, scroll='1m')

In [200]:
def rest(res):
    scrollId = res['_scroll_id']
    new_res = es.scroll(scroll_id = scrollId, scroll = '1m')
    return new_res

In [132]:
def add(res):
    a = []
    for i in res['hits']['hits']:
        a.append(i['_id'])
    return a

In [211]:
res1['hits']['hits']

[{'_index': 'btc_top100_index',
  '_type': 't',
  '_id': 'y5gTyGUB3n6arYnVKAYB',
  '_score': 1.0,
  '_source': {'Block': 540929,
   'Time (UTC)': '2018-09-11T09:21:04Z',
   'year': 2018,
   'month': 9,
   'date': 11,
   'timestamp': 20180911,
   'Amount (BTC)': -77.42391931,
   'Balance (BTC)': 11241.86510256,
   'Balance (USD)': 71246520.0,
   'USD price': 6337.61,
   'address': '1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s',
   'rank': 93}},
 {'_index': 'btc_top100_index',
  '_type': 't',
  '_id': '1ZgTyGUB3n6arYnVKAYB',
  '_score': 1.0,
  '_source': {'Block': 540917,
   'Time (UTC)': '2018-09-11T07:08:12Z',
   'year': 2018,
   'month': 9,
   'date': 11,
   'timestamp': 20180911,
   'Amount (BTC)': 13.35781251,
   'Balance (BTC)': 11046.19099424,
   'Balance (USD)': 69943791.0,
   'USD price': 6331.94,
   'address': '1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s',
   'rank': 93}},
 {'_index': 'btc_top100_index',
  '_type': 't',
  '_id': '3pgTyGUB3n6arYnVKAYB',
  '_score': 1.0,
  '_source': {'Block': 5409

In [217]:
res2 = rest(res1)
res3 = rest(res2)
res4 = rest(res3)
res5 = rest(res4)
res6 = rest(res5)
res7 = rest(res6)
res8 = rest(res7)

In [262]:
def es_all(Host='35.194.152.180', Index="btc_top100_index"):
    total = []
    es = Elasticsearch([Host], timeout=30)
    doc = {'size':10000, 'query':{"match_all": {}}}
    
    # first
    res1 = es.search(index=Index, body=doc, scroll='1m')
    total.append(res1['hits']['hits'])
    # second
    res_ = rest(res1)
    total.append(res_['hits']['hits'])
    # third and then
    while res_['hits']['hits'] != []:
        res_ = rest(res_)
        if res_['hits']['hits'] != []:
            total.append(res_['hits']['hits'])
    
    doc = total[0]
    for i in range(1, len(total)):
        doc += total[i]
    data = list(map(lambda x:x['_source'], doc))
    return data

In [266]:
es_all(Index='dataframe')

[{'A': 45, 'B': 9, 'C': 62, 'D': 69},
 {'A': 67, 'B': 82, 'C': 46, 'D': 88},
 {'A': 83, 'B': 0, 'C': 50, 'D': 36},
 {'A': 64, 'B': 70, 'C': 78, 'D': 58},
 {'A': 91, 'B': 16, 'C': 29, 'D': 17},
 {'A': 73, 'B': 9, 'C': 55, 'D': 75},
 {'A': 60, 'B': 7, 'C': 47, 'D': 14},
 {'A': 18, 'B': 97, 'C': 49, 'D': 37},
 {'A': 28, 'B': 12, 'C': 30, 'D': 21},
 {'A': 31, 'B': 92, 'C': 79, 'D': 30},
 {'A': 58, 'B': 33, 'C': 3, 'D': 46},
 {'A': 71, 'B': 34, 'C': 62, 'D': 1},
 {'A': 69, 'B': 88, 'C': 95, 'D': 82},
 {'A': 3, 'B': 13, 'C': 73, 'D': 72},
 {'A': 80, 'B': 79, 'C': 26, 'D': 28},
 {'A': 52, 'B': 35, 'C': 12, 'D': 11},
 {'A': 77, 'B': 5, 'C': 50, 'D': 62},
 {'A': 79, 'B': 31, 'C': 76, 'D': 72},
 {'A': 26, 'B': 35, 'C': 63, 'D': 28},
 {'A': 68, 'B': 42, 'C': 9, 'D': 63},
 {'A': 74, 'B': 55, 'C': 68, 'D': 79},
 {'A': 36, 'B': 56, 'C': 35, 'D': 72},
 {'A': 37, 'B': 35, 'C': 40, 'D': 73},
 {'A': 83, 'B': 0, 'C': 50, 'D': 36},
 {'A': 27, 'B': 48, 'C': 21, 'D': 18},
 {'A': 74, 'B': 50, 'C': 4, 'D': 47

In [232]:
for i in test:
    print(len(i))

10000
10000
10000
10000
10000
10000
5319


In [242]:
doc = test[0]
for i in range(1, len(test)):
    doc += test[i]
print(len(doc))

65319


In [254]:
d = list(map(lambda x:x['_source'], doc))

In [251]:
t.to_json(orient='records')

'[{"Block":538049,"Time (UTC)":"2018-08-23T03:34:37Z","year":2018,"month":8,"date":23,"timestamp":20180823,"Amount (BTC)":-49.10277715,"USD price":6411.63,"address":"1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s","rank":70},{"Block":538048,"Time (UTC)":"2018-08-23T03:24:18Z","year":2018,"month":8,"date":23,"timestamp":20180823,"Amount (BTC)":-11.64232808,"USD price":6411.63,"address":"1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s","rank":70},{"Block":538047,"Time (UTC)":"2018-08-23T03:13:43Z","year":2018,"month":8,"date":23,"timestamp":20180823,"Amount (BTC)":-64.8441401,"USD price":6411.63,"address":"1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s","rank":70},{"Block":538046,"Time (UTC)":"2018-08-23T02:52:07Z","year":2018,"month":8,"date":23,"timestamp":20180823,"Amount (BTC)":49.09049688,"USD price":6411.63,"address":"1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s","rank":70},{"Block":538045,"Time (UTC)":"2018-08-23T02:39:42Z","year":2018,"month":8,"date":23,"timestamp":20180823,"Amount (BTC)":-4.77728105,"USD price":6411.63,"add

In [271]:
def json_pandas(data):
    import json
    str_d = json.dumps(data)
    df = pd.read_json(str_d, orient='records')
    return df

In [1]:
from pd_ela import *
df = json_pandas(es_all()).sort_values('Time (UTC)', ascending=False)

In [2]:
df.head()

,Amount (BTC),Balance (BTC),Balance (USD),Block,Time (UTC),USD price,address,date,month,rank,timestamp,year
13248,853.251505,12881.899967,74146774,550115,2018-11-14T23:50:16Z,5755.89,16CCZHtjTH4GppotnBvwhkoYXtXaSKTzmQ,14,11,86,20181114,2018
13249,0.000032,57560.373972,331311066,550114,2018-11-14T23:29:46Z,5755.89,34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo,14,11,14,20181114,2018
26344,-14627.407900,0.000000,0,550110,2018-11-14T23:14:23Z,5755.89,3Bo9FTs5Fx8tXoUKJZeDDFXbvpfNK18qQ1,14,11,76,20181114,2018
52340,177.884547,12028.648462,68692725,550102,2018-11-14T22:10:33Z,5710.76,16CCZHtjTH4GppotnBvwhkoYXtXaSKTzmQ,14,11,86,20181114,2018
65370,-2000.000965,22266.049532,126642774,550101,2018-11-14T22:05:11Z,5687.71,3Kzh9qAqVWQhEsfQz7zEQL1EuSx5tyNLNS,14,11,58,20181114,2018


In [3]:
df.to_csv('f100.csv', index=False)

In [7]:
df.shape

(65373, 12)

In [1]:
from pd_ela import *

In [8]:
e.time_filtered('2018-11-17T09:21:04Z')

In [9]:
res = e.get_res()
doc = res['hits']['hits']
data = list(map(lambda x:x['_source'], doc))
pd.DataFrame(data).sort_values('Time (UTC)', ascending=False)

,Amount (BTC),Balance (BTC),Balance (USD),Block,Time (UTC),USD price,address,date,month,rank,timestamp,year
166,249.999000,15199.778890,54128376.0,554292,2018-12-18T01:48:48Z,3561.13,15xZvLJLAgxh8cxEmAbpx1H35oibHo5cLA,18,12,53,20181218,2018
165,0.000008,53000.013395,188739895.0,554292,2018-12-18T01:48:48Z,3561.13,1JCe8z4jJVNXSjohjM4i9Hh813dLCNx2Sy,18,12,11,20181218,2018
334,75.741397,30950.221579,107718710.0,554198,2018-12-17T12:52:38Z,3480.39,336xGpGweq1wtY4kRTuA4w6d7yDkBU9czU,17,12,30,20181217,2018
335,137.072174,30874.480183,105428439.0,554183,2018-12-17T11:27:14Z,3414.74,336xGpGweq1wtY4kRTuA4w6d7yDkBU9czU,17,12,30,20181217,2018
404,0.000010,107203.076508,353435612.0,554136,2018-12-17T03:05:18Z,3296.88,16rCmCmbuWDhPjWTrpQGaU3EPdZF7MTdUk,17,12,5,20181217,2018
164,249.999000,14949.779890,49155852.0,554127,2018-12-17T01:45:11Z,3288.07,15xZvLJLAgxh8cxEmAbpx1H35oibHo5cLA,17,12,53,20181217,2018
405,0.000641,138660.862010,456873626.0,554114,2018-12-16T23:23:41Z,3294.90,3D2oetdNuZUqQHPJmcMDDHYoqkyNVsFk9r,16,12,1,20181216,2018
68,0.000010,107848.285071,354520994.0,554078,2018-12-16T14:54:11Z,3287.22,3Nxwenay9Z8Lc9JBiywExpnEFiLp6Afp8v,16,12,4,20181216,2018
252,249.999000,14699.780890,48272534.0,554013,2018-12-16T01:44:23Z,3283.89,15xZvLJLAgxh8cxEmAbpx1H35oibHo5cLA,16,12,53,20181216,2018
66,249.999000,14449.781890,47409448.0,553865,2018-12-15T01:43:53Z,3280.98,15xZvLJLAgxh8cxEmAbpx1H35oibHo5cLA,15,12,53,20181215,2018


In [12]:
e.query = {
            "size":0,
            "aggs" : {"max_time" : { "max" : { "field" : "Time (UTC)" } }
            }
        }

In [13]:
e.get_res()

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 65487, 'max_score': 0.0, 'hits': []},
 'aggregations': {'max_time': {'value': 1542959270000.0,
   'value_as_string': '2018-11-23T07:47:50.000Z'}}}

In [10]:
q = {
        "bool": {"filter": {
            "range": {
              'Time (UTC)': {
                "gt": '2018-09-11T09:21:04Z'
              }
            }
          }
        }
    }
t = es.search(index="btc_top100_index", body={'size':10000, 'query':q})

In [11]:
t

{'took': 87,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 6042,
  'max_score': 0.0,
  'hits': [{'_index': 'btc_top100_index',
    '_type': 't',
    '_id': 'Hpk1FWYB3n6arYnVQZTu',
    '_score': 0.0,
    '_source': {'Block': 543136,
     'Time (UTC)': '2018-09-26T07:44:53Z',
     'year': 2018,
     'month': 9,
     'date': 26,
     'timestamp': 20180926,
     'Amount (BTC)': -64.06445891,
     'Balance (BTC)': 15841.5610847,
     'Balance (USD)': 102166494.0,
     'USD price': 6449.27,
     'address': '1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s',
     'rank': 72}},
   {'_index': 'btc_top100_index',
    '_type': 't',
    '_id': 'K5k1FWYB3n6arYnVQZTu',
    '_score': 0.0,
    '_source': {'Block': 543116,
     'Time (UTC)': '2018-09-26T05:33:37Z',
     'year': 2018,
     'month': 9,
     'date': 26,
     'timestamp': 20180926,
     'Amount (BTC)': -10.91133325,
     'Balance (BTC)': 15947.38767065,
     'Balance (USD)': 102681636.0,
  

In [2]:
data = es_all()
df = json_pandas(data)
df_tech = technical(df)

In [5]:
df_tech.head(50)

,es_date,timestamp,btc_volume,btc_price,RSI_15,K_9,D_9,Vola_30
timestamp,,,,,,,,
2015-01-04,2015-01-04T00:00:00Z,20150104,186.460,282.470,NaN,NaN,NaN,NaN
2015-01-05,2015-01-05T00:00:00Z,20150105,50.000,269.970,NaN,NaN,NaN,NaN
2015-01-06,2015-01-06T00:00:00Z,20150106,550.000,277.801,NaN,NaN,NaN,NaN
2015-01-07,2015-01-07T00:00:00Z,20150107,1594.033,285.063,NaN,NaN,NaN,NaN
2015-01-08,2015-01-08T00:00:00Z,20150108,100.000,286.555,NaN,NaN,NaN,NaN
2015-01-09,2015-01-09T00:00:00Z,20150109,300.000,284.535,NaN,NaN,NaN,NaN
2015-01-10,2015-01-10T00:00:00Z,20150110,4800.000,281.093,NaN,NaN,NaN,NaN
2015-01-11,2015-01-11T00:00:00Z,20150111,600.000,273.157,NaN,NaN,NaN,NaN
2015-01-12,2015-01-12T00:00:00Z,20150112,222.598,268.430,NaN,NaN,NaN,NaN


In [10]:
df_tech.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1418 entries, 2015-01-04 to 2018-12-18
Data columns (total 8 columns):
es_date          1418 non-null object
timestamp        1418 non-null int64
btc_volume       1418 non-null float64
btc_price        1418 non-null float64
RSI_15           1403 non-null float64
K_9              1406 non-null float64
D_9              1406 non-null float64
Volatility_30    1388 non-null float64
dtypes: float64(6), int64(1), object(1)
memory usage: 99.7+ KB


In [5]:
es.indices.delete(index='btc_tech', ignore=[400, 404])

{'acknowledged': True}

In [8]:
res = es.indices.create(index = 'btc_tech')
print(" response: '%s'" % (res))

 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'btc_tech'}'


In [4]:
esbulk('btc_tech', 'indicator', df_tech)

In [5]:
es.search(index="btc_tech", body={'query':{"match_all": {}}}
         )

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 1388,
  'max_score': 1.0,
  'hits': [{'_index': 'btc_tech',
    '_type': 'indicator',
    '_id': 'rY9owGcBsVV8gDWLmlre',
    '_score': 1.0,
    '_source': {'es_date': '2015-02-04T00:00:00Z',
     'raw_date': 20150204,
     'btc_volume': 1747.45488912,
     'btc_price': 226.18200000000002,
     'RSI_15': 49.67228783887785,
     'K_9': 2.509363379645731,
     'D_9': 12.204209106813456,
     'Volatility_30': 97.28288423816882}},
   {'_index': 'btc_tech',
    '_type': 'indicator',
    '_id': 'so9owGcBsVV8gDWLmlrf',
    '_score': 1.0,
    '_source': {'es_date': '2015-02-09T00:00:00Z',
     'raw_date': 20150209,
     'btc_volume': 208.1855386,
     'btc_price': 221.048,
     'RSI_15': 48.19404068847975,
     'K_9': 3.830857210854006,
     'D_9': 1.860348006471283,
     'Volatility_30': 89.7803846068791}},
   {'_index': 'btc_tech',
    '_type': 'indicator',
    '_id': 'to9

In [7]:
es.delete_by_query(index='btc_tech',doc_type='indicator', body={'query':{"match_all": {}}})

{'took': 342,
 'timed_out': False,
 'total': 1388,
 'deleted': 1388,
 'batches': 2,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [12]:
def update_tech(Index='btc_tech', docType='indicator'):
    df_tech = technical(json_pandas(es_all()))
    es.delete_by_query(index=Index, doc_type=docType, body={'query':{"match_all": {}}})
    res = esbulk(Index, docType, df_tech)

In [11]:
print(update())

None


In [6]:
update_tech()

In [4]:
e = Ela_result(es, index="btc_tech")

In [5]:
e.query={'query':{"match_all": {}}}
e.get_res()

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 1388,
  'max_score': 1.0,
  'hits': [{'_index': 'btc_tech',
    '_type': 'indicator',
    '_id': '2Y-OwGcBsVV8gDWL2HB2',
    '_score': 1.0,
    '_source': {'es_date': '2015-02-08T00:00:00Z',
     'timestamp': 20150208,
     'btc_volume': 8.88388629,
     'btc_price': 226.83,
     'RSI_15': 47.935795738306794,
     'K_9': 0.009679703855787203,
     'D_9': 1.8684707617743828,
     'Vola_30': 88.82951051112585}},
   {'_index': 'btc_tech',
    '_type': 'indicator',
    '_id': '4Y-OwGcBsVV8gDWL2HB9',
    '_score': 1.0,
    '_source': {'es_date': '2015-02-16T00:00:00Z',
     'timestamp': 20150216,
     'btc_volume': 112.95172769999999,
     'btc_price': 233.64333333333335,
     'RSI_15': 48.06204359148769,
     'K_9': 26.58662109304144,
     'D_9': 23.657042399496323,
     'Vola_30': 86.20092537866023}},
   {'_index': 'btc_tech',
    '_type': 'indicator',
    '_id': '44-O

In [7]:
e.query

{'match_all': {}}